In [1]:
from constants import *
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import re
import pickle
from gensim.models import Word2Vec, Doc2Vec
from train_w2v import EpochLogger, EpochSaver

pd.options.display.max_rows = 500
path = 'NETL/additional_support_files'


-----

### re-implementation of 'pruned_documents.py'

In [2]:
dewiki_phrases_lemmatized = 'dewiki_phrases_lemmatized.pickle'
dpl = pd.read_pickle(join(ETL_PATH, dewiki_phrases_lemmatized))

In [3]:
dpl

,sent_idx,text,token,doc_len,title_len
hash,,,,,
8952056961092092653,1,Alan_Smithee,Alan_Smithee,747,2
598046625986755870,44,Actinium,Actinium,866,1
8442369265370766621,98,Ang_Lee,ANG_Lee,1632,2
-5325279570187525080,181,Anschluss,Anschluss,413,1
5107548614255273253,199,Aussagenlogik,Aussagenlogik,7211,1
-6810310479569543740,599,Anthony_Minghella,Anthony_Minghella,637,2
-291419119128528545,636,US-amerikanischer_Film,us-amerikanisch_Film,2992,2
-686601136003585762,767,Vorsätze_für_Maßeinheiten,Vorsatz_für_Maßeinheit,992,3
-928617659304474122,829,Abkürzungen/Gesetze_und_Recht,Abkürzungen/Gesetz_und_Recht,167,3


In [7]:
title_length = 4 # filter < 5
doc_length = 40 # filter > 40

In [8]:
# creating a list containing original and lemmatized phrases
dplx = dpl.query('doc_len >= @doc_length and title_len <= @title_length')

In [9]:
dplx

,sent_idx,text,token,doc_len,title_len
hash,,,,,
8952056961092092653,1,Alan_Smithee,Alan_Smithee,747,2
598046625986755870,44,Actinium,Actinium,866,1
8442369265370766621,98,Ang_Lee,ANG_Lee,1632,2
-5325279570187525080,181,Anschluss,Anschluss,413,1
5107548614255273253,199,Aussagenlogik,Aussagenlogik,7211,1
-6810310479569543740,599,Anthony_Minghella,Anthony_Minghella,637,2
-291419119128528545,636,US-amerikanischer_Film,us-amerikanisch_Film,2992,2
-686601136003585762,767,Vorsätze_für_Maßeinheiten,Vorsatz_für_Maßeinheit,992,3
-928617659304474122,829,Abkürzungen/Gesetze_und_Recht,Abkürzungen/Gesetz_und_Recht,167,3


In [4]:
lst_lemma = x.text.append(x.token).tolist()
lst_lemma = sorted(set(lst_lemma))

In [ ]:
output_file = 'short_label_documents_de_lemmatized'
outpath = join(path, output_file)
print("Writing labels to pickle file", outpath)
with open(outpath, 'wb') as fp:
    pickle.dump(lst_lemma, fp)

In [5]:
len(lst_lemma), lst_lemma

(1778215,
 ['!_!_!',
  '!_Distain',
  '!_distain',
  '$_50SAT',
  '&_RQ',
  '&_Radieschen',
  '(_)',
  '(_.)p(_..._)nin',
  '(_030_)_Magazin',
  '(_1,5-Cyclooctadien)(1,3,5-cyclooctatrien)ruthenium',
  '(_100000_)_Astronautica',
  '(_10000_)_Myriostos',
  '(_10001_)_Palermo',
  '(_100027_)_Hannaharendt',
  '(_100033_)_Taizé',
  '(_10007_)_Malytheatre',
  '(_1000_)_Piazzia',
  '(_10010_)_Rudruna',
  '(_100133_)_Demosthenes',
  '(_10015_)_Valenlebedev',
  '(_1001_)_Gaussia',
  '(_100267_)_JAXA',
  '(_100268_)_Rosenthal',
  '(_1002_)_Olbersia',
  '(_10030_)_Philkeenan',
  '(_10038_)_Tanaro',
  '(_1003_)_Lilofee',
  '(_10041_)_Parkinson',
  '(_10044_)_Squyres',
  '(_100483_)_NAOJ',
  '(_1004_)_Belopolskya',
  '(_10055_)_Silcher',
  '(_10057_)_L’Obel',
  '(_1005_)_Arago',
  '(_100604_)_Lundy',
  '(_10067_)_Bertuch',
  '(_10068_)_Dodoens',
  '(_10069_)_Fontenelle',
  '(_1006_)_Lagrangea',
  '(_10071_)_Paraguay',
  '(_10072_)_Uruguay',
  '(_10079_)_Meunier',
  '(_1007_)_Pawlowia',
  '(_10088_

-----

### re-implementation of 'get_indices.py'

In [5]:
d2vpath = join(ETL_PATH, 'embeddings', 'd2v', 'd2v')
w2vpath = join(ETL_PATH, 'embeddings', 'w2v', 'w2v')
print("Doc2Vec loading", d2vpath)
d2v = Doc2Vec.load(d2vpath)

Doc2Vec loading ../data/preprocessed/embeddings/d2v/d2v


In [8]:
print("Word2Vec loading", w2vpath)
w2v = Word2Vec.load(w2vpath)
print("models loaded")

Word2Vec loading ../data/preprocessed/embeddings/w2v/w2v
models loaded


In [6]:
len(d2v.docvecs.doctags), d2v.docvecs.doctags

(3928652,
 {'8952056961092092653': Doctag(offset=0, word_count=1104, doc_count=2),
  'Alan_Smithee': Doctag(offset=1, word_count=1104, doc_count=2),
  '598046625986755870': Doctag(offset=2, word_count=1434, doc_count=2),
  'Actinium': Doctag(offset=3, word_count=1434, doc_count=2),
  '8442369265370766621': Doctag(offset=4, word_count=2526, doc_count=2),
  'ANG_Lee': Doctag(offset=5, word_count=1263, doc_count=1),
  'Ang_Lee': Doctag(offset=6, word_count=1263, doc_count=1),
  '-5325279570187525080': Doctag(offset=7, word_count=690, doc_count=2),
  'Anschluss': Doctag(offset=8, word_count=1196, doc_count=6),
  '5107548614255273253': Doctag(offset=9, word_count=11286, doc_count=2),
  'Aussagenlogik': Doctag(offset=10, word_count=11286, doc_count=2),
  '-6810310479569543740': Doctag(offset=11, word_count=914, doc_count=2),
  'Anthony_Minghella': Doctag(offset=12, word_count=914, doc_count=2),
  '-291419119128528545': Doctag(offset=13, word_count=5036, doc_count=2),
  'us-amerikanisch_Film'

In [7]:
len(d2v.wv.vocab), d2v.wv.vocab

(1344667,
 {'Alan_Smithee': <gensim.models.keyedvectors.Vocab at 0x7f9a96ba4e80>,
  'stehen': <gensim.models.keyedvectors.Vocab at 0x7f9a96ba4f60>,
  'als': <gensim.models.keyedvectors.Vocab at 0x7f9a96ba4fd0>,
  'Pseudonym': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae048>,
  'für': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae080>,
  'ein': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae0f0>,
  'fiktiv': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae160>,
  'Regisseur': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae198>,
  'der': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae208>,
  'Film': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae278>,
  'verantworten': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae2b0>,
  'bei': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae320>,
  'eigentlich': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae358>,
  'seinen': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae3c8>,
  'Name': <gensim.models.keyedvectors.Vocab at 0x7f9a96bae438

In [9]:
len(w2v.wv.vocab), w2v.wv.vocab

(726359,
 {'Alan_Smithee': <gensim.models.keyedvectors.Vocab at 0x7f9a96ba4e48>,
  'stehen': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d9438>,
  'als': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d94e0>,
  'Pseudonym': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d9518>,
  'für': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d95c0>,
  'ein': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d9588>,
  'fiktiv': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d96a0>,
  'Regisseur': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d9630>,
  'der': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d96d8>,
  'Film': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d9748>,
  'verantworten': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d97f0>,
  'bei': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d97b8>,
  'eigentlich': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d9828>,
  'seinen': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d9908>,
  'Name': <gensim.models.keyedvectors.Vocab at 0x7f9a7f8d98d0>

In [11]:
in_file = 'short_label_documents_de_lemmatized'
inpath = join(path, in_file)
with open(inpath, 'rb') as fp:
    labels_de = pickle.load(fp)

labels_de

['!_!_!',
 '!_Distain',
 '!_distain',
 '$_50SAT',
 '&_RQ',
 '&_Radieschen',
 '(_)',
 '(_.)p(_..._)nin',
 '(_030_)_Magazin',
 '(_1,5-Cyclooctadien)(1,3,5-cyclooctatrien)ruthenium',
 '(_100000_)_Astronautica',
 '(_10000_)_Myriostos',
 '(_10001_)_Palermo',
 '(_100027_)_Hannaharendt',
 '(_100033_)_Taizé',
 '(_10007_)_Malytheatre',
 '(_1000_)_Piazzia',
 '(_10010_)_Rudruna',
 '(_100133_)_Demosthenes',
 '(_10015_)_Valenlebedev',
 '(_1001_)_Gaussia',
 '(_100267_)_JAXA',
 '(_100268_)_Rosenthal',
 '(_1002_)_Olbersia',
 '(_10030_)_Philkeenan',
 '(_10038_)_Tanaro',
 '(_1003_)_Lilofee',
 '(_10041_)_Parkinson',
 '(_10044_)_Squyres',
 '(_100483_)_NAOJ',
 '(_1004_)_Belopolskya',
 '(_10055_)_Silcher',
 '(_10057_)_L’Obel',
 '(_1005_)_Arago',
 '(_100604_)_Lundy',
 '(_10067_)_Bertuch',
 '(_10068_)_Dodoens',
 '(_10069_)_Fontenelle',
 '(_1006_)_Lagrangea',
 '(_10071_)_Paraguay',
 '(_10072_)_Uruguay',
 '(_10079_)_Meunier',
 '(_1007_)_Pawlowia',
 '(_10088_)_Digne',
 '(_10089_)_Turgot',
 '(_10093_)_Diesel',
 '

In [14]:
doc_indices_de = []
word_indices_de = []

dout = []
wout = []
for label in labels_de[:]:
    try:
        idx = d2v.docvecs.doctags[label].offset
        doc_indices_de.append(idx)
    except:
        dout.append(label)
    try:
        idx = w2v.wv.vocab[label].index
        word_indices_de.append(idx)
    except:
        wout.append(label)
        # print(f'{label} not in w2v vocab')

doc_indices_de

[1238454,
 1335977,
 1335978,
 1553847,
 3334610,
 1593182,
 2546821,
 2749106,
 2694746,
 1668679,
 1849185,
 1864667,
 1473164,
 3173770,
 3770256,
 1473244,
 3111580,
 1505757,
 1587132,
 1473270,
 2766432,
 2318096,
 1266738,
 2810311,
 1509277,
 1505301,
 2742839,
 1509945,
 1512664,
 2318871,
 2766466,
 1283650,
 809034,
 2766494,
 1410476,
 1513383,
 1514385,
 809501,
 2766517,
 2440671,
 2440693,
 809525,
 2810327,
 1519975,
 1519882,
 1520016,
 2444489,
 2743718,
 577713,
 2743767,
 1828856,
 1829317,
 2743924,
 1829328,
 2743986,
 1829416,
 1829418,
 2806108,
 807491,
 2743639,
 2743600,
 2743494,
 2806210,
 2742969,
 3884551,
 687498,
 311583,
 2742944,
 3109712,
 1450362,
 1447064,
 835686,
 2742872,
 2641284,
 1451333,
 2810340,
 3292211,
 2744152,
 510700,
 1733896,
 510855,
 1405828,
 509821,
 1733876,
 2806967,
 509957,
 517742,
 1732985,
 3277258,
 2744345,
 2744484,
 2809168,
 2810920,
 2811018,
 994982,
 2766625,
 2769224,
 2809187,
 2811037,
 3105094,
 991545,
 1375

In [15]:
# most of the labels are OOV, probably due to min_count=20 (that parametrization is from the NETL implementation)
len(doc_indices_de), len(dout), len(word_indices_de), len(wout)

(1778215, 0, 311768, 1466447)

In [16]:
doc2vec_indices_file_de = join(ETL_PATH, 'embeddings', 'doc2vec_indices_de')
word2vec_indices_file_de = join(ETL_PATH, 'embeddings', 'word2vec_indices_de')

# creating output indices file
with open(doc2vec_indices_file_de, 'wb') as fp:
    pickle.dump(doc_indices_de, fp)
with open(word2vec_indices_file_de, 'wb') as fp:
    pickle.dump(word_indices_de, fp)

-----
### Pagerank titles -> remove categories from title phrases

In [ ]:
path = '../../master_cloud/external_code/NETL-Automatic-Topic-Labelling-/model_run/support_files/de/'
df = pd.read_csv(join(path, 'pagerank-titles-sorted_de.txt'), sep='\t', header=None, keep_default_na=False)

def process_titles(title):
    title = title.split('(')[0].strip('_')
    return title

df[1] = df[1].map(process_titles)
df.to_csv(join(path, 'pagerank-titles-sorted_de_categories_removed.txt'), sep='\t', header=None, index=None)